# Estimation of hydraulic propeties using moddeling tools

### Project GRANAR - MECHA

To mesure the root hydraulic conductivities, experimental procedures are rather challenging and time consuming. Here is a srcipt that assembles the Generator of Root Anatomy in R [GRANAR](https://granar.github.io/) and the Model of Explicit Cross-section Hydraulic Architecture [MECHA](https://mecharoot.github.io/).
Together it is possible to generate a root cross-section and estimates its hydraulic properties, such as: it's radial hydraulic conductivity (kr) and the axial hydraulic conductance (kx). A few maturation senario are implemented inside the model to mirror the different apoplastic barrier developpement along the root axis. 

## GRANAR:
Upon a small set of anatomical features the GRANAR model is able to reconstruct a generic root cell network for mono and dicotyledon.

Change kernel to R to launch the following script.


In [2]:
# Loading R library
library(tidyverse)
library(plyr)
library(deldir)
library(sp)
library(xml2)
library(viridis)

source("./GRANAR/R/granar.R")

In [ ]:
# Load one parameter file for GRANAR
params <- read_param_xml("GRANAR/model_params/Zea_mays_1_Heymans_2019.xml")

 # # # # # # # # # # # # # # #
#  To change paramter value   #
 # # # # # # # # # # # # # # #
    
# Xylem size (diameter)
params$value[params$type == "max_size" & params$name == "xylem"] <- 0.043

# aerenchyma proportion
params$value[params$type == "proportion" & params$name == "aerenchyma"] <- 0.2
# number of lacuna
params$value[params$type == "n_files" & params$name == "aerenchyma"] <- 15

# Generate the anatomy
sim <- create_anatomy(parameters = params, verbatim=T, paraview = F)
# sim is a list
# sim$nodes is a data frame with most of the information
# sim$output is a data frame with summarize anatomical features

In [ ]:
# To visualize the anatomy and the scenario that are going to be tested.
# you can use the plot_anatomy function.
plot_anatomy(sim, col = "segment", apo_bar = 1)
plot_anatomy(sim, col = "segment", apo_bar = 2)
plot_anatomy(sim, col = "segment", apo_bar = 3)

# To visualize cell type:
plot_anatomy(sim) # default type

In [ ]:
# write geometry
write_anatomy_xml(sim, "./MECHA/cellsetdata/current_root.xml")
# give explicit location of root lacuna in geometry
aer_in_geom_xml(sim, path = "./MECHA/Projects/granar/in/Maize_Geometry.xml")

In [ ]:
# hydraulic conductivity of standard walls
kw = 2.4E-4 # [cm^2/hPa/d] Zhu and Steudle (1991)

# Cell membrane permeability, with separate contribution of the biphospholipid layer (km) and AQP (kAQP)
km = 3.0E-5 # [cm/hPa/d]  after removal of kAQP and kpl from Elhert et al. (2009) and Bret-Hart and Silk (1994)
kAQP = 4.3E-4 # [cm/hPa/d] uniformly distributed on each cell layer

# Individual plasmodesma conductance
kpl = 5.3E-12 #  [cm^3/hPa/d/plasmodesmata] Geometrical average from Bret-Hart and Silk (1994)

# change cell hydraulic properties
microhydro(path = "MECHA/Projects/granar/in/Maize_Hydraulics.xml",
           kw, km, kAQP, kpl)

## MECHA:
Upon this generated cross-section and the cell hydraulic properties defined in Hydraulic.xml
MECHA estimates the radial hydraulic conductivities (for three scenarios: 1 = an endodermal casparian strip, 2 = a fully suberized endodermis, 3 = fully suberized endodermis and a casparian strip on the exodermis.

Change kernel to Python to launch the following script. Once the kernel is changed, all variables stored in R are gone.

In [ ]:
from mecha_function import *

mecha()

This last script cell might take a 2 to 5 min to run and gives the estimated hydraulic properties.

To visualize the results change again the kernel to R. Sorry about that, my python skills are not that good.


In [ ]:
# Loading again the R library
library(tidyverse)
library(plyr)
library(deldir)
library(sp)
library(xml2)
library(viridis)

source("./GRANAR/R/granar.R")

Upload MECHA output 

In [ ]:
Flux1 <- ApoSymp("MECHA/Projects/granar/out/Root/Project_Test/baseline/Macro_prop_1,0.txt")
Flux2 <- ApoSymp("MECHA/Projects/granar/out/Root/Project_Test/baseline/Macro_prop_2,1.txt")
Flux3 <- ApoSymp("MECHA/Projects/granar/out/Root/Project_Test/baseline/Macro_prop_4,2.txt")

Plot the results to see the symplastic water fluxes (in blue) and the apoplastic fluxes (in light brown). The sum of the two water fluxes equals to the total water fluxes. Once the water is in the stele, the water is being lifted by the xylem poles.

In [ ]:
plot_water_flux(Flux1)
plot_water_flux(Flux2)
plot_water_flux(Flux3)